In [1]:
# @title Environment Setup
import os
import sys

# 1. *** FIX: Clear problematic environment variable for matplotlib ***
# This prevents the "ValueError: Key backend: 'module://matplotlib_inline.backend_inline'" error
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/requirements.txt
!uv pip install gdown
# 3. Install PyTorch 1.13
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
!uv pip uninstall torch torchvision
!uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 15052, done.
remote: Counting objects: 100% (2865/2865), done.
remote: Compressing objects: 100% (2792/2792), done.
remote: Total 15052 (delta 92), reused 2838 (delta 71), pack-reused 12187 (from 3)
Receiving objects: 100% (15052/15052), 246.86 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (480/480), done.
Updating files: 100% (110/110), done.
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 112ms
Prepared 1 package in 106ms
Uninstalled 1 package in 68ms
Installed 1 package in 16ms
 - pip==24.1.2
 + pip==25.3
Using Python 3.12.12 environment at: /usr
Resolved 116 packages in 659ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1)

      [stdout]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.linux-x86_64-cpython-312/t

In [1]:
import os
from IPython import get_ipython

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


In [2]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dungngocpham171 (dungngocpham171-university-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
import gdown

if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

In [4]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -q -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [5]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
# Create the checkpoint directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
# Wait loop to check if files exist
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]

while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]

    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt

✅ All weights found! You can proceed to the next step.


In [25]:
import pandas as pd
nom_tu_tao_300_df = pd.read_csv(f"{INPUT_PATH}/Ds_300_ChuNom_TuTao.csv")
nom_tu_tao = nom_tu_tao_300_df['word'].tolist()

with open(f"{OUTPUT_PATH}/nom_tu_tao.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(nom_tu_tao))
kieu = [𤾓,𢆥,𥪞,揆,𠊛,些,𡦂,才,𡦂,命,窖,𱺵,恄,饒,𣦆,戈,没,局,𣷭,橷,仍,調,𬖉,𧡊,㐌,𤴬,疸,𢚸,𨔍,之,彼,嗇,斯,豊,𡗶,撑,涓,貝,𦟐,紅,打,㭴,稿,𦹳,吝,𱠎,𠓀,畑,風,情,固,錄,群,傳,史,撑,浪,𢆥,󰞸,靖,朝,明,𦊚,方,𪹚,𣼽,𠄩,京,凭,鐄,固,茹,員,外,户,王,家,資,擬,拱,常,常,堛,中]
print(",".join(kieu))

SyntaxError: invalid non-printable character U+F07B8 (ipython-input-1462724732.py, line 7)

In [27]:
%cd FontDiffusion
!python sample_optimized.py \
    --ckpt_dir="../ckpt/" \
    --style_image_path="/content/FontDiffusion/DVSKTT_ref.jpg" \
    --save_image \
    --ttf_path="/content/FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --character_input \
    --content_character="𤾓,𢆥,𥪞,揆,𠊛,些,𡦂,才,𡦂,命,窖,𱺵,恄,饒,𣦆,戈,没,局,𣷭,橷,仍,調,𬖉,𧡊,㐌,𤴬,疸,𢚸,𨔍,之,彼,嗇,斯,豊,𡗶,撑,涓,貝,𦟐,紅,打,㭴,稿,𦹳,吝,𱠎,𠓀,畑,風,情,固,錄,群,傳,史,撑,浪,𢆥,󰞸,靖,朝,明,𦊚,方,𪹚,𣼽,𠄩,京,凭,鐄,固,茹,員,外,户,王,家,資,擬,拱,常,常,堛,中" \
    --save_image_dir="../kieu_output/" \
    --device="cuda:0" \
    --algorithm_type="dpmsolver++" \
    --guidance_type="classifier-free" \
    --guidance_scale=7.5 \
    --num_inference_steps=20 \
    --method="multistep" \
    --channels_last

[Errno 2] No such file or directory: 'FontDiffusion'
/content/FontDiffusion
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER - OPTIMIZED SAMPLING
Model: ../ckpt/
Device: cuda:0
FP16: False
Channels Last: True
Batch Size: 1

Loading FontDiffuser pipeline...
Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  DownBlock2D
Load the up block  UpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  UpBlock2D
Param count for Ds initialized parameters: 20591296
Get CG-GAN Style Encoder!
Param count for Ds initialized param

In [28]:
                                                        # @title Zipping the results folder
!zip -r {OUTPUT_PATH}/kieu_output.zip {OUTPUT_PATH}/kieu_output
print(f"Finish zipped the output data, ready for downloading")

  adding: content//kieu_output/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/橷/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/橷/out_single.png (deflated 0%)
  adding: content//kieu_output/NomNaTong-Regular/𠄩/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/𠄩/out_single.png (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/常/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/常/out_single.png (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/畑/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/畑/out_single.png (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/𪹚/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/𪹚/out_single.png (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/情/ (stored 0%)
  adding: content//kieu_output/NomNaTong-Regular/情/out_single.png (stored 0%)
  adding: content//kieu_output/NomNaTong

In [24]:
# @title Happy Christmas✨
# !rm -r -f FontDiffusion